In [0]:
# ІМПОРТ ДАНИХ

events_df = spark.read.option("header", True).option("inferSchema", True) \
    .csv("/Volumes/workspace/default/retailrocket_dataset/events.csv")

events_df.show(5, truncate=False)


+-------------+---------+-----+------+-------------+
|timestamp    |visitorid|event|itemid|transactionid|
+-------------+---------+-----+------+-------------+
|1433221332117|257597   |view |355908|NULL         |
|1433224214164|992329   |view |248676|NULL         |
|1433221999827|111016   |view |318965|NULL         |
|1433221955914|483717   |view |253185|NULL         |
|1433221337106|951259   |view |367447|NULL         |
+-------------+---------+-----+------+-------------+
only showing top 5 rows


In [0]:
# ОЗНАЙОМЛЕННЯ ЗІ СТРУКТРОЮ ДАНИХ

from pyspark.sql.functions import col

print("Схема датасету:")
events_df.printSchema()

print("Рядки та стовпці:")
print((events_df.count(), len(events_df.columns)))


Схема датасету:
root
 |-- timestamp: long (nullable = true)
 |-- visitorid: integer (nullable = true)
 |-- event: string (nullable = true)
 |-- itemid: integer (nullable = true)
 |-- transactionid: integer (nullable = true)

Рядки та стовпці:
(2756101, 5)


In [0]:
# ПЕРВИННИЙ АНАЛІЗ ЯКОСТІ ДАНИХ

from pyspark.sql.functions import col, count, when, min, max, avg

# Підрахунок пропусків у кожній колонці
print("Кількість пропусків у кожній колонці:")
events_df.select([count(when(col(c).isNull(), c)).alias(c) for c in events_df.columns]).show()

# Виявлення дублікатів
print("Кількість дублікатів:")
duplicates_count = events_df.groupBy("timestamp", "visitorid", "itemid").count().filter("count > 1").count()
print(duplicates_count)

# Перевірка категоріальних змінних (унікальні значення)
categorical_columns = ["event"]
for c in categorical_columns:
    print(f"Унікальні значення в колонці '{c}':")
    events_df.select(c).distinct().show(truncate=False)

# Перевірка часових полів на аномалії (мін та макс timestamp)
date_columns = ["timestamp"]
for c in date_columns:
    print(f"Часові значення в колонці '{c}': min та max")
    events_df.select(min(c).alias("min_timestamp"), max(c).alias("max_timestamp")).show(truncate=False)

# Первинна перевірка числових полів
numeric_columns = [c[0] for c in events_df.dtypes if c[1] in ("int", "double")]
if numeric_columns:
    print("Аналіз числових колонок:")
    events_df.select(
        [min(c).alias(f"{c}_min") for c in numeric_columns] +
        [max(c).alias(f"{c}_max") for c in numeric_columns] +
        [avg(c).alias(f"{c}_avg") for c in numeric_columns]
    ).show(truncate=False)

Кількість пропусків у кожній колонці:
+---------+---------+-----+------+-------------+
|timestamp|visitorid|event|itemid|transactionid|
+---------+---------+-----+------+-------------+
|        0|        0|    0|     0|      2733644|
+---------+---------+-----+------+-------------+

Кількість дублікатів:
468
Унікальні значення в колонці 'event':
+-----------+
|event      |
+-----------+
|addtocart  |
|view       |
|transaction|
+-----------+

Часові значення в колонці 'timestamp': min та max
+-------------+-------------+
|min_timestamp|max_timestamp|
+-------------+-------------+
|1430622004384|1442545187788|
+-------------+-------------+

Аналіз числових колонок:
+-------------+----------+-----------------+-------------+----------+-----------------+-----------------+-----------------+-----------------+
|visitorid_min|itemid_min|transactionid_min|visitorid_max|itemid_max|transactionid_max|visitorid_avg    |itemid_avg       |transactionid_avg|
+-------------+----------+-----------------

In [0]:
# ОЧИЩЕННЯ ДАНИХ

from pyspark.sql.functions import col, when, from_unixtime, to_timestamp, year, month, dayofmonth, dayofweek, weekofyear, lit

# Видалення дублікатів
events_cleaned = events_df.dropDuplicates(["timestamp", "visitorid", "itemid"])
print("Рядки після видалення дублікатів:", events_cleaned.count())

# Заповнення пропусків у transactionid: якщо немає транзакції, то ставимо 0
events_cleaned = events_cleaned.withColumn(
    "transactionid",
    when(col("transactionid").isNull(), lit(0))
    .otherwise(col("transactionid"))
)

# Фільтрація некоректних подій: залишаємо тільки відомі типи
valid_events = ["view", "addtocart", "transaction"]
events_cleaned = events_cleaned.filter(col("event").isin(valid_events))

# Перевірка та приведення timestamp у формат datetime
events_cleaned = events_cleaned.withColumn(
    "event_time",
    to_timestamp(from_unixtime(col("timestamp")/1000))  # timestamp у мс → datetime
)



Рядки після видалення дублікатів: 2755630


In [0]:
 #СТАНДАРТИЗАЦІЯ ТА ТРАНСФОРМАЦІЯ

# Розбиття timestamp на складові
events_transformed = events_cleaned.withColumn("year", year(col("event_time"))) \
                                   .withColumn("month", month(col("event_time"))) \
                                   .withColumn("day", dayofmonth(col("event_time"))) \
                                   .withColumn("weekday", dayofweek(col("event_time"))) \
                                   .withColumn("weekofyear", weekofyear(col("event_time")))

# Категоріальне кодування event у числовий індекс
event_mapping = {"view": 0, "addtocart": 1, "transaction": 2}
events_transformed = events_transformed.withColumn(
    "event_indexed",
    when(col("event") == "view", 0)
    .when(col("event") == "addtocart", 1)
    .when(col("event") == "transaction", 2)
    .otherwise(-1)
)

display(events_transformed.limit(10))


timestamp,visitorid,event,itemid,transactionid,event_time,year,month,day,weekday,weekofyear,event_indexed
1433193207640,628886,view,105274,0,2015-06-01T21:13:27.000Z,2015,6,1,2,23,0
1433200504091,982569,view,182473,0,2015-06-01T23:15:04.000Z,2015,6,1,2,23,0
1433208091217,1373098,view,243353,0,2015-06-02T01:21:31.000Z,2015,6,2,3,23,0
1433143168495,78096,view,365014,0,2015-06-01T07:19:28.000Z,2015,6,1,2,23,0
1433203256537,172329,view,18423,0,2015-06-02T00:00:56.000Z,2015,6,2,3,23,0
1433194773521,731686,view,228068,0,2015-06-01T21:39:33.000Z,2015,6,1,2,23,0
1433192073436,122149,view,228996,0,2015-06-01T20:54:33.000Z,2015,6,1,2,23,0
1433199404336,487963,view,272421,0,2015-06-01T22:56:44.000Z,2015,6,1,2,23,0
1433179867135,286616,view,303715,0,2015-06-01T17:31:07.000Z,2015,6,1,2,23,0
1433163526262,1213495,view,380237,0,2015-06-01T12:58:46.000Z,2015,6,1,2,23,0


In [0]:
# ЗБЕРЕЖЕННЯ РЕЗУЛЬТАТУ
delta_path = "/Volumes/workspace/default/retail_rocket_delta_table"

events_transformed.write.format("delta") \
    .mode("overwrite") \
    .save(delta_path)

print(f"Датасет успішно збережено у Delta Lake за шляхом: {delta_path}")


Датасет успішно збережено у Delta Lake за шляхом: /Volumes/workspace/default/retail_rocket_delta_table
